In [82]:
import os
import numpy as np

data_dir = '/home/dfc13/WaveformDeepLearning/data/SPEs/d1_shifted/'
times = []
ADCs = []
for filename in os.listdir(data_dir)[:9]:
    if filename.endswith(".txt"):
        times.append([])
        ADCs.append([])
        filename = os.path.join(data_dir, filename)
        with open(filename) as f:
            lines = f.readlines()

#        lines = data.split('\n')
        # Extract just the lines with (t,Q) data
        lines = lines[5:]
#
        for line in lines:
            if not line == '':
                # append data with time in ns and ADC value ranging from -1 to 1
                times[-1].append(float(line.split(',')[0])*1.e9)
                ADCs[-1].append(float(line.split(',')[1])/40.e-3)
targets = np.full(len(ADCs),1.0) # if it is an SPE, label it with a "1"

In [83]:
times = np.array(times)
ADCs = np.array(ADCs)
targets = np.array(targets)

In [84]:
print times.shape,ADCs.shape, targets.shape
print targets
#print times,ADCs
print ADCs.min(),ADCs.max(),ADCs.max()-ADCs.min()
print times.min(),times.max(),times.max()-times.min()

(9, 3950) (9, 3950) (9,)
[1. 1. 1. 1. 1. 1. 1. 1. 1.]
-0.29109999999999997 0.072575 0.36367499999999997
-28.7 174.0 202.7


In [85]:
data_dir = '/home/dfc13/WaveformDeepLearning/data/SPEs/not_spe/'
ns_times = []
ns_ADCs = []
for filename in os.listdir(data_dir)[:9]:
    if filename.endswith(".txt"):
        ns_times.append([])
        ns_ADCs.append([])
        filename = os.path.join(data_dir, filename)
        with open(filename) as f:
            lines = f.readlines()

#        lines = data.split('\n')
        # Extract just the lines with (t,Q) data
        lines = lines[5:]
#
        for line in lines:
            if not line == '':
                # append data with time in ns and ADC value ranging from -1 to 1
                ns_times[-1].append(float(line.split(',')[0])*1.e9)
                ns_ADCs[-1].append(float(line.split(',')[1])/40.e-3)
ns_targets = np.full(len(ns_ADCs),0.0) # if it is an SPE, label it with a "1"

In [86]:
ns_times = np.array(ns_times)
ns_ADCs = np.array(ns_ADCs)
ns_targets = np.array(ns_targets)

In [87]:
print ns_times.shape,ns_ADCs.shape,ns_targets.shape
print ns_targets
#print ns_times,ns_ADCs
print ns_ADCs.min(),ns_ADCs.max(),ns_ADCs.max()-ns_ADCs.min()
print ns_times.min(),ns_times.max(),ns_times.max()-ns_times.min()

(9, 3950) (9, 3950) (9,)
[0. 0. 0. 0. 0. 0. 0. 0. 0.]
-0.033075 0.031175 0.06425
466.90744 664.3902400000001 197.48280000000005


In [93]:
all_targets = []
all_targets = np.array(all_targets)
all_targets = np.concatenate((targets,ns_targets),axis=0)
#
all_ADCs = []
all_ADCs = np.array(all_ADCs)
all_ADCs = np.concatenate((ADCs,ns_ADCs),axis=0)

In [95]:
print all_ADCs.shape,ADCs.shape,ns_ADCs.shape
print all_ADCs,"\n",ADCs,"\n",ns_ADCs
print all_targets.shape,targets.shape,ns_targets.shape
print all_targets,"\n",targets,"\n",ns_targets

 (18, 3950) (9, 3950) (9, 3950)
[[ 1.8750e-03  1.9250e-03  2.0500e-03 ...  5.9575e-02  6.6000e-02
   7.2575e-02]
 [-1.0000e-03 -9.2500e-04 -8.5000e-04 ... -1.0200e-02 -1.1925e-02
  -1.3675e-02]
 [ 1.1500e-03  1.0250e-03  8.5000e-04 ... -1.0575e-02 -1.1875e-02
  -1.3200e-02]
 ...
 [ 6.2750e-03  6.5750e-03  6.8750e-03 ...  1.7850e-02  1.9375e-02
   2.0925e-02]
 [-6.0000e-04 -5.0000e-04 -4.0000e-04 ... -2.0000e-04 -1.2500e-04
  -7.5000e-05]
 [ 2.8750e-03  2.7500e-03  2.6000e-03 ...  6.1750e-03  5.6500e-03
   5.1000e-03]] 
[[ 1.8750e-03  1.9250e-03  2.0500e-03 ...  5.9575e-02  6.6000e-02
   7.2575e-02]
 [-1.0000e-03 -9.2500e-04 -8.5000e-04 ... -1.0200e-02 -1.1925e-02
  -1.3675e-02]
 [ 1.1500e-03  1.0250e-03  8.5000e-04 ... -1.0575e-02 -1.1875e-02
  -1.3200e-02]
 ...
 [-3.7750e-03 -4.2500e-03 -4.7000e-03 ...  5.0250e-03  5.7250e-03
   6.4750e-03]
 [-1.0000e-04 -1.0000e-04 -1.0000e-04 ...  1.1000e-02  1.2150e-02
   1.3325e-02]
 [ 3.5750e-03  3.4250e-03  3.2750e-03 ... -1.3500e-03 -2.2250e-03

Do I need to merge time & ADC for each set of data, or can I just use the ADC data?
Create array of vectors containing (target,time+ADC)
Mix them up to avoid lack of "commutativity" in DNN

In [96]:
from sklearn.model_selection import train_test_split
ADCs_train, ADCs_test, targets_train, targets_test = train_test_split(all_ADCs, all_targets, test_size=0.33, random_state=42)

In [97]:
print ADCs_train, targets_train
print "-----"
print ADCs_test, targets_test

[[-6.0000e-04 -5.0000e-04 -4.0000e-04 ... -2.0000e-04 -1.2500e-04
  -7.5000e-05]
 [ 6.2750e-03  6.5750e-03  6.8750e-03 ...  1.7850e-02  1.9375e-02
   2.0925e-02]
 [ 5.7000e-03  5.9000e-03  6.0500e-03 ... -2.8025e-02 -3.0525e-02
  -3.3075e-02]
 ...
 [-1.7000e-03 -1.6000e-03 -1.5250e-03 ...  3.9750e-03  4.8500e-03
   5.7500e-03]
 [ 2.5000e-03  2.4500e-03  2.4000e-03 ... -7.7250e-03 -8.4500e-03
  -9.2000e-03]
 [-3.7750e-03 -4.2500e-03 -4.7000e-03 ...  5.0250e-03  5.7250e-03
   6.4750e-03]] [0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1.]
-----
[[ 0.001875  0.001925  0.00205  ...  0.059575  0.066     0.072575]
 [-0.001    -0.000925 -0.00085  ... -0.0102   -0.011925 -0.013675]
 [ 0.003575  0.003425  0.003275 ... -0.00135  -0.002225 -0.00315 ]
 [ 0.003975  0.004175  0.004375 ...  0.011375  0.012925  0.0145  ]
 [-0.002225 -0.002625 -0.002975 ...  0.0056    0.005875  0.006125]
 [ 0.002925  0.00285   0.00275  ...  0.02665   0.0289    0.031175]] [1. 1. 1. 1. 1. 0.]
